检查每个claim，第一个检索结果如果是 factcheck.afp.com 或者 reuters.com ，
然后再核对title是否对应的上claim：
找claim第一个单词，再title里找到对应的起始位置，然后终止位置为...
查看title是否在claim中出现，如果是，那么FALSE改为TRUE


In [1]:
import json
import sys
dirs = ["../..", "../create_prompt_llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
import prompt_rag

search_engine = "brave"

def load_train_bak():
    with open("covmis_data/train.json.bak", "r") as f:
        return json.load(f)
    
def load_train():
    with open("covmis_data/train.json", "r") as f:
        return json.load(f)
    
def load_type1():
    with open("output/type_1.json", "r") as f:
        return json.load(f)

def load_type2():
    with open("output/type_2.json", "r") as f:
        return json.load(f)
    
def save_type1(x):
    with open("output/type_1.json", "w") as f:
        json.dump(x, f, indent=4)

def item_exist(x, data):
    for item in data:
        if item["id"] == x["id"]:
            return True
    return False

def update_data_type1(x, data: list):

    data_type2 = load_type2()
    if item_exist(x, data_type2):
        return
    
    if not item_exist(x, data):
        data.append(x)

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)
try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)

def save_diff(x, model_name, diff_version):
    with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
        json.dump(x, f, indent=4)

def load_diff(model_name, diff_version):
    try:
        with open(f"output/diff_{model_name}_v{diff_version}.json", "r") as f:
            return json.load(f)
    except:
        data_init = [{"claim": i["claim"], "label": int(i["label"])} for i in data_search_llm]
        with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
            json.dump(data_init, f, indent=4)
        return data_init


In [2]:
cnt = 0
for item in data_search:
    claim = item["claim"]
    title = item[f"{search_engine}_search_results"]["web"]["results"][0]["title"]
    url = item[f"{search_engine}_search_results"]["web"]["results"][0]["url"]
    check_list = ["poynter"] # 

    ok = False
    for site in check_list:
        if site in url:
            ok = True
            break
    if not ok:
        continue
    cnt += 1

    print(claim)
    break
cnt


Dr Mikovits, an established scientist, revealed that Dr Anthony Fauci was paid to cover up the extent of the problem caused by COVID-19. Also she revealed that SARS-CoV-2 was created in a lab and that influenza vaccines increase COVID-19 mortality by 36%.


1

In [2]:
s = "Summary of the given information:\n\nInformation 1, published on 2020-07-25, is an article from a fact-checking organization called El Sabueso. It verifies a press conference from a group called 'Doctors for the truth' and finds their claims about cures, prevention, and the spread of the virus to be false or out of context.\n\nInformation 2, published on 2022-02-14, is a Johns Hopkins Medicine article that addresses common myths about COVID-19. However, it does not specifically mention 'Doctors for the truth' or any of their claims.\n\nInformation 3, with no specific publication date, is an article discussing the role of physicians and researchers in combating misinformation about COVID-19. It highlights their efforts to use blogs, social media, and conversations to guide people towards safe practices.\n\nInformation 4, published on 2017-10-19, is a Pew Research Center article discussing the future of truth and misinformation online. It features opinions from experts about the potential for reducing misinformation online, but it does not specifically mention COVID-19 or 'Doctors for the truth.'\n\nInformation 5, published on 2021-09-20, is an article discussing whether doctors spreading COVID-19 misinformation will ever face consequences. It mentions that some top spreaders of misinformation are physicians with active licenses, but it does not specifically mention 'Doctors for the truth.'\n\nBased on the available information, the claim is:\nTRUE\n\nReasoning:\nThe claim, published on 2020-07-25, states that a group called 'Doctors for the truth' shared false or out of context data regarding cures, prevention, and the spread of the virus. Information 1, published on the same day, confirms this claim by fact-checking the press conference and finding the doctors' arguments to be false or without scientific evidence. The other information sources do not contradict this claim and do not provide any evidence that would change the classification."





print(s)


Summary of the given information:

Information 1, published on 2020-07-25, is an article from a fact-checking organization called El Sabueso. It verifies a press conference from a group called 'Doctors for the truth' and finds their claims about cures, prevention, and the spread of the virus to be false or out of context.

Information 2, published on 2022-02-14, is a Johns Hopkins Medicine article that addresses common myths about COVID-19. However, it does not specifically mention 'Doctors for the truth' or any of their claims.

Information 3, with no specific publication date, is an article discussing the role of physicians and researchers in combating misinformation about COVID-19. It highlights their efforts to use blogs, social media, and conversations to guide people towards safe practices.

Information 4, published on 2017-10-19, is a Pew Research Center article discussing the future of truth and misinformation online. It features opinions from experts about the potential for re

添加related url

In [15]:
data_train = load_train()

for item in data_train:
    related_url = ""
    article_id = item["related_articles"][-1]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/articles/{article_id}.txt", "r") as f:
        lines = f.readlines()
    
    s = ""
    now = len(lines) - 1
    while now >=0 and lines[now].strip == "":
        now -= 1

    s = lines[now]
    st = s.find("article_url:")
    if st != -1:
        related_url = s[st + len("article_url:"):].strip()
    item["related_url"] = related_url
    

# with open("covmis_data/train.json", "w") as f:
#     json.dump(data_train, f, indent=4)



In [7]:
diff_data = load_diff("llama3", 1)
data_train = load_train()
data_type1 = load_type1()
data_type2 = load_type2()

claim_dict = {}
for item in diff_data:
    if item["match2"].lower() == "no":
        claim_dict[item["claim"].strip()] = 1

cnt = 0
cnt2 = 0
for i, item in enumerate(data_search):
    claim = item["claim"]
    if int(data_train[i]["label"]) == 1:
        continue
    # ok = False
    for search_result_id in range(2):
        if len(item[f"{search_engine}_search_results"]["web"]["results"]) == search_result_id:
            break
        title_bak = item[f"{search_engine}_search_results"]["web"]["results"][search_result_id]["title"]
        title = title_bak
        url = item[f"{search_engine}_search_results"]["web"]["results"][search_result_id]["url"]
        check_list = [
            "factcheck.afp.com", "reuters.com", 
            "africacheck.org"
        ]

        ok = False
        for site in check_list:
            if site in url:
                ok = True
                break
        if not ok:
            continue

        # cnt += 1
        pos_st = title.find(claim.split(' ')[0])
        if pos_st == -1:
            continue
        pos_en = len(title)
        if title.endswith(" ..."):
            pos_en -= 4
        elif title.endswith("..."):
            pos_en -= 3 

        title = title[pos_st : pos_en]
        pos_en = len(title)

        # ["| Reuters", "| Fact Check", "| Fact", "|"]
        pos_verical_bar = title.rfind("|")
        if pos_verical_bar != -1:
            pos_en = pos_verical_bar

        if len(title[:pos_en].split()) >= 5 and title[:pos_en].strip() in claim:
            if claim_dict.get(claim.strip()) is not None:
                cnt += 1
                # print(claim)
                if data_train[i]["label"] == 0 and not item_exist(data_train[i], data_type2):
                    data_train[i]["label"] = 2
                    update_data_type1(data_train[i], data_type1)
                    # print(claim)
                else:
                    pass
            else:
                cnt2 += 1
                
                if data_train[i]["label"] == 0:
                    
                    if not title_bak.lower().startswith("false claim"):
                        if (title_bak.lower().startswith("fact check:") and "reuters.com" in url) or \
                            ("africacheck.org" in url):
                            data_train[i]["label"] = 2
                            update_data_type1(data_train[i], data_type1)
                            # print(claim)
                        else:
                            pass
                else:
                    pass
            break

cnt, cnt2, cnt + cnt2


Video makes several misleading claims over COVID-19


(609, 154, 763)

In [4]:
data_type1 = load_type1()
data_type2 = load_type2()

len(data_type1) + len(data_type2)


673

In [4]:
# save_type1(data_type1)

# with open("covmis_data/train.json", "w") as f:
#     json.dump(data_train, f, indent=4)

# len(data_type1)


703